In [1]:
## THis script is to extract PWV DETERMINISTIC RUN 
#from sites using bilinear interpolation version 02
## modified from Sam Shohan (16 November 2023) and translated from MATLAB (JMM and CBR 2019)
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import random
import pandas as pd
import xarray as xr
from scipy.interpolate import griddata
import gzip
from netCDF4 import Dataset,num2date
import time
import matplotlib.pyplot as plt
import matplotlib as mplt
import cartopy.crs as ccrs
import cartopy.feature as cfea
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cartopy import feature as cf
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import matplotlib.colors
import os 
import glob
from get_PWVgrid import get_griddedPWV, get_griddedPWV_abs
def read_nx(ncfile):
    dx = xr.open_dataset(ncfile)
    pwv = dx.PWV[:,:]
    return pwv
##### Define the bilinear interpolation function #######################################
def bilinear_interpolation(x, y, lon, lat, data):
    points = np.column_stack((lon.values.ravel(), lat.values.ravel()))
    values = data.ravel()
    interp_values = griddata(points, values, (x, y), method='linear')
    return interp_values

In [2]:
## get data here
dir_in = '20210815_06Z_40mem_0035_6cyc_vertisundef_assim'

wrf_path = '/net/stratus/st6/bayu/ARIZONA_PRO/DART_deterministic/'+dir_in+'/'
wrf_files = glob.glob(wrf_path + 'wrfout_d02*')
dir_out = '/home/bayu/Documents/ARIZONA-WRF/wrf_out/'+dir_in+'/'

# Read station information from the text file
sta_data = []

pathmeta = '/net/stratus/st6/bayu/ARIZONA_PRO/metadata/'
with open(pathmeta + 'GPS_DAVID-SUOMI_sites_2021.txt', 'r') as station_file:
    for line in station_file:
        parts = line.split()
        if len(parts) >= 3:  # Ensure there's at least one word after splitting
            lon, lat, hgt, name = parts
            lon = float(lon)
            lat = float(lat)
            name = str(name)
            height = float(hgt)
            sta_data.append((lon, lat, height, name))

In [3]:
print(len(wrf_files))

19


In [4]:
# define the output file
output_file = dir_out + 'GPSequiv_DETR_sta_pwv.dat'

## Loop through the files
for cycle in range(len(wrf_files)):
    data = xr.open_dataset(wrf_files[cycle])
    time = data.Times.values
    timee = time.astype('U19')
    PWV,XLON,XLAT = get_griddedPWV_abs(wrf_files[cycle])
    print('working on '+wrf_files[cycle])
    with open(output_file, 'a') as outfile:
            for lon, lat, height, name in sta_data:
                pwv = bilinear_interpolation(lon, lat, XLON, XLAT, PWV)
                pwv_value = pwv  # Assign the interpolated value directly
                outfile.write(f"{name},{height},{lon},{lat},{timee},{pwv_value}\n")

working on /net/stratus/st6/bayu/ARIZONA_PRO/DART_deterministic/20210815_06Z_40mem_0035_6cyc_vertisundef_assim/wrfout_d02_2021-08-15_18:00:00
working on /net/stratus/st6/bayu/ARIZONA_PRO/DART_deterministic/20210815_06Z_40mem_0035_6cyc_vertisundef_assim/wrfout_d02_2021-08-15_19:00:00
working on /net/stratus/st6/bayu/ARIZONA_PRO/DART_deterministic/20210815_06Z_40mem_0035_6cyc_vertisundef_assim/wrfout_d02_2021-08-15_20:00:00
working on /net/stratus/st6/bayu/ARIZONA_PRO/DART_deterministic/20210815_06Z_40mem_0035_6cyc_vertisundef_assim/wrfout_d02_2021-08-15_21:00:00
working on /net/stratus/st6/bayu/ARIZONA_PRO/DART_deterministic/20210815_06Z_40mem_0035_6cyc_vertisundef_assim/wrfout_d02_2021-08-15_22:00:00
working on /net/stratus/st6/bayu/ARIZONA_PRO/DART_deterministic/20210815_06Z_40mem_0035_6cyc_vertisundef_assim/wrfout_d02_2021-08-15_23:00:00
working on /net/stratus/st6/bayu/ARIZONA_PRO/DART_deterministic/20210815_06Z_40mem_0035_6cyc_vertisundef_assim/wrfout_d02_2021-08-16_00:00:00
workin